In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold
from lightgbm import LGBMClassifier
import os

## Definition of path and downloading the data
As sometimes I run the notebook on kaggle and sometimes on my PC I must change the path.

In [2]:
#global variables
KAGGLE = 1
PATH = "/kaggle/input/" if KAGGLE ==1 else "C:/Users/pzlpa_000/"

In [3]:
df_train = pd.read_csv( os.path.join(PATH,"isic-2024-challenge/train-metadata.csv"))
df_test = pd.read_csv( os.path.join(PATH,"isic-2024-challenge/test-metadata.csv"))

/tmp/ipykernel_18/3060065471.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv( os.path.join(PATH,"isic-2024-challenge/train-metadata.csv"))


In [4]:
df_train.dtypes

isic_id                          object
target                            int64
patient_id                       object
age_approx                      float64
sex                              object
anatom_site_general              object
clin_size_long_diam_mm          float64
image_type                       object
tbp_tile_type                    object
tbp_lv_A                        float64
tbp_lv_Aext                     float64
tbp_lv_B                        float64
tbp_lv_Bext                     float64
tbp_lv_C                        float64
tbp_lv_Cext                     float64
tbp_lv_H                        float64
tbp_lv_Hext                     float64
tbp_lv_L                        float64
tbp_lv_Lext                     float64
tbp_lv_areaMM2                  float64
tbp_lv_area_perim_ratio         float64
tbp_lv_color_std_mean           float64
tbp_lv_deltaA                   float64
tbp_lv_deltaB                   float64
tbp_lv_deltaL                   float64


In [5]:
# for checking
#df_test=df_train[200000:]
#df_train=df_train[:200000]

## Explonatory DF
I create a df with columns description to consult it if necessary inside the notebook.

In [6]:
data = {
    "field_name": [
        "target", "lesion_id", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", 
        "mel_mitotic_index", "mel_thick_mm", "tbp_lv_dnn_lesion_confidence", 
        "isic_id", "patient_id", "age_approx", "sex", "anatom_site_general", 
        "clin_size_long_diam_mm", "image_type", "tbp_tile_type", "tbp_lv_A", "tbp_lv_Aext", 
        "tbp_lv_B", "tbp_lv_Bext", "tbp_lv_C", "tbp_lv_Cext", "tbp_lv_H", "tbp_lv_Hext", 
        "tbp_lv_L", "tbp_lv_Lext", "tbp_lv_areaMM2", "tbp_lv_area_perim_ratio", 
        "tbp_lv_color_std_mean", "tbp_lv_deltaA", "tbp_lv_deltaB", "tbp_lv_deltaL", 
        "tbp_lv_deltaLBnorm", "tbp_lv_eccentricity", "tbp_lv_location", 
        "tbp_lv_location_simple", "tbp_lv_minorAxisMM", "tbp_lv_nevi_confidence", 
        "tbp_lv_norm_border", "tbp_lv_norm_color", "tbp_lv_perimeterMM", 
        "tbp_lv_radial_color_std_max", "tbp_lv_stdL", "tbp_lv_stdLExt", 
        "tbp_lv_symm_2axis", "tbp_lv_symm_2axis_angle", "tbp_lv_x", "tbp_lv_y", "tbp_lv_z", 
        "attribution", "copyright_license"
    ],
    "description": [
        "Binary class {0: benign, 1: malignant}.", 
        "Unique lesion identifier. Present in lesions that were manually tagged as a lesion of interest.", 
        "Fully classified lesion diagnosis.", 
        "First level lesion diagnosis.", 
        "Second level lesion diagnosis.", 
        "Third level lesion diagnosis.", 
        "Fourth level lesion diagnosis.", 
        "Fifth level lesion diagnosis.", 
        "Mitotic index of invasive malignant melanomas.", 
        "Thickness in depth of melanoma invasion.", 
        "Lesion confidence score (0-100 scale).", 
        "Unique case identifier.", 
        "Unique patient identifier.", 
        "Approximate age of patient at time of imaging.", 
        "Sex of the person.", 
        "Location of the lesion on the patient's body.", 
        "Maximum diameter of the lesion (mm).", 
        "Structured field of the ISIC Archive for image type.", 
        "Lighting modality of the 3D TBP source image.", 
        "A inside lesion.", 
        "A outside lesion.", 
        "B inside lesion.", 
        "B outside lesion.", 
        "Chroma inside lesion.", 
        "Chroma outside lesion.", 
        "Hue inside the lesion; calculated as the angle of A* and B* in LAB* color space. Typical values range from 25 (red) to 75 (brown).", 
        "Hue outside lesion.", 
        "L inside lesion.", 
        "L outside lesion.", 
        "Area of lesion (mm^2).", 
        "Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.", 
        "Color irregularity, calculated as the variance of colors within the lesion's boundary.", 
        "Average A contrast (inside vs. outside lesion).", 
        "Average B contrast (inside vs. outside lesion).", 
        "Average L contrast (inside vs. outside lesion).", 
        "Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta LB of the lesion relative to its immediate background in LAB* color space. Typical values range from 5.5 to 25.", 
        "Eccentricity.", 
        "Classification of anatomical location, divides arms & legs to upper & lower; torso into thirds.", 
        "Classification of anatomical location, simple.", 
        "Smallest lesion diameter (mm).", 
        "Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.", 
        "Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.", 
        "Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.", 
        "Perimeter of lesion (mm).", 
        "Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in LAB* color space within concentric rings originating from the lesion center. Values range 0-10.", 
        "Standard deviation of L inside lesion.", 
        "Standard deviation of L outside lesion.", 
        "Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.", 
        "Lesion border asymmetry angle.", 
        "X-coordinate of the lesion on 3D TBP.", 
        "Y-coordinate of the lesion on 3D TBP.", 
        "Z-coordinate of the lesion on 3D TBP.", 
        "Image attribution, synonymous with image source.", 
        "Copyright license."
    ],
    "train_only": [
        True, True, True, True, True, True, True, True, True, True, True,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False
    ]
}

df_explained = pd.DataFrame(data)


In [7]:
pd.set_option('display.max_colwidth', None)
print(df_explained['description'][df_explained['field_name']=="tbp_lv_color_std_mean"])

31    Color irregularity, calculated as the variance of colors within the lesion's boundary.
Name: description, dtype: object


In [8]:
#creating a variable with columns that appear both in test and train
test_and_target=["target", 
        "isic_id", "patient_id", "age_approx", "sex", "anatom_site_general", 
        "clin_size_long_diam_mm", "image_type", "tbp_tile_type", "tbp_lv_A", "tbp_lv_Aext", 
        "tbp_lv_B", "tbp_lv_Bext", "tbp_lv_C", "tbp_lv_Cext", "tbp_lv_H", "tbp_lv_Hext", 
        "tbp_lv_L", "tbp_lv_Lext", "tbp_lv_areaMM2", "tbp_lv_area_perim_ratio", 
        "tbp_lv_color_std_mean", "tbp_lv_deltaA", "tbp_lv_deltaB", "tbp_lv_deltaL", 
        "tbp_lv_deltaLBnorm", "tbp_lv_eccentricity", "tbp_lv_location", 
        "tbp_lv_location_simple", "tbp_lv_minorAxisMM", "tbp_lv_nevi_confidence", 
        "tbp_lv_norm_border", "tbp_lv_norm_color", "tbp_lv_perimeterMM", 
        "tbp_lv_radial_color_std_max", "tbp_lv_stdL", "tbp_lv_stdLExt", 
        "tbp_lv_symm_2axis", "tbp_lv_symm_2axis_angle", "tbp_lv_x", "tbp_lv_y", "tbp_lv_z", 
        "attribution", "copyright_license"
    ]

In [9]:
#creating a df with columns that appear both in train and test
df_train_filtered=df_train[test_and_target]
categorical_columns = df_train_filtered.select_dtypes(include=['object', 'category']).columns
numerical_columns = df_train_filtered.select_dtypes(include=['number']).columns

## EDA


In [10]:
# checking columns with zero values as they can be really null values
def value_counts (df, value):
    value_columns = df.columns[(df == value).any()]
    print(value_columns)

    # Count number of 0s in each column that has at least one 0
    value_counts = df[value_columns].eq(0).sum()

    # Print the number of 0s in each relevant column
    print(f"Number of {value} in each column that has at least one {value}:{value_counts}")

value_counts(df_train_filtered,0)
 

Index(['target', 'tbp_lv_color_std_mean', 'tbp_lv_nevi_confidence',
       'tbp_lv_norm_color', 'tbp_lv_radial_color_std_max',
       'tbp_lv_symm_2axis_angle'],
      dtype='object')
Number of 0 in each column that has at least one 0:target                         400666
tbp_lv_color_std_mean           26187
tbp_lv_nevi_confidence              6
tbp_lv_norm_color               26182
tbp_lv_radial_color_std_max     29733
tbp_lv_symm_2axis_angle         18562
dtype: int64


In [11]:
# Calculate mean for numerical columns grouped by 'target'
means_by_target = df_train_filtered[numerical_columns].groupby(df_train_filtered['target']).mean()
print(means_by_target)

# Check the number of different values in cat columns to see which cat columns can be useful
for c in categorical_columns:
    num_dif = df_train_filtered[c].nunique()
    if num_dif < 30:
        print (f'{c}: {num_dif} different values : {df_train_filtered[c].unique()}')
    else:
        print (f'{c}: {num_dif} different values')             
    

        target  age_approx  clin_size_long_diam_mm   tbp_lv_A  tbp_lv_Aext  \
target                                                                       
0          0.0   58.009694                3.929043  19.971528    14.916682   
1          1.0   61.371795                5.749771  22.500924    17.534401   

         tbp_lv_B  tbp_lv_Bext   tbp_lv_C  tbp_lv_Cext   tbp_lv_H  ...  \
target                                                             ...   
0       28.286065    26.914940  34.788027    30.921401  54.661450  ...   
1       23.837896    24.950406  33.067349    30.796881  46.741528  ...   

        tbp_lv_norm_color  tbp_lv_perimeterMM  tbp_lv_radial_color_std_max  \
target                                                                       
0                3.090164           11.872182                     1.015873   
1                4.545228           18.718538                     1.613218   

        tbp_lv_stdL  tbp_lv_stdLExt  tbp_lv_symm_2axis  \
target             

In [12]:
# Check distribution of one cat variable for target 0 and 1
place_by_target = df_train_filtered.groupby(['anatom_site_general', 'target']).size().unstack(fill_value=0)
place_totals = place_by_target.sum(axis=1)
place_by_target['percentage_1'] = (place_by_target[1] / place_totals) * 100
print(place_by_target)

target                    0    1  percentage_1
anatom_site_general                           
anterior torso        87688   82      0.093426
head/neck             11968   78      0.647518
lower extremity      102955   73      0.070855
posterior torso      121799  103      0.084494
upper extremity       70500   57      0.080786


## Preprocessing

In [13]:
df_train_filtered = df_train_filtered[df_train_filtered['anatom_site_general'].notna()]
df_train_filtered.reset_index(drop=True, inplace=True)

In [15]:
def feature_engineering_process(df, num_columns, medians):
     # Handle numerical columns missing values
    for nc in num_columns:
        df[nc] = df[nc].fillna(medians.get(nc, 0))  
    # Replacement for 0 to na to avoid incorrect calulation in feature engineering as 0 really mean missing value
    replace_cols = ['tbp_lv_color_std_mean', 'tbp_lv_norm_color', 'tbp_lv_radial_color_std_max']
    for col in replace_cols:
        if col in df.columns:
            df[col] = df[col].replace(0, np.nan)
    #Feature engineering
    df_num = df[num_columns]
    df_num = feature_engineering(df_num)   
    numerical_columns_list = list(df_num.columns)    
    
    return df_num, numerical_columns_list

In [16]:
class Preprocessor:
    def __init__(self):
        self.medians = {}
        self.one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
        self.scaler = StandardScaler()
        self.encoded_cols = []
        self.numerical_columns = []
        self.cat_cols = ["sex", "anatom_site_general"]       

    def fit(self, df):
        # Check if input is a numpy array and convert to DataFrame if necessary
        if isinstance(df, np.ndarray):
            df = pd.DataFrame(df)
        
        # Determine numerical columns
        self.numerical_columns = list(df.select_dtypes(include=['number']).columns)       
        if "target" in self.numerical_columns:
            self.numerical_columns.remove("target") 
        
        # Compute medians for numerical columns
        num_medians = df[self.numerical_columns].median()
        self.medians = dict(num_medians)

        # Compute median for angle
        if 'tbp_lv_symm_2axis_angle' in df.columns:
            self.medians['tbp_lv_symm_2axis_angle'] = df['tbp_lv_symm_2axis_angle'].median()
        
        # Fit OneHotEncoder
        df['sex'] = df['sex'].fillna('male')
        df[self.cat_cols] = df[self.cat_cols].fillna('other') 
        self.one_hot_encoder.fit(df[self.cat_cols])
        self.encoded_cols = self.one_hot_encoder.get_feature_names_out(self.cat_cols)                 
        
        # I have to do feature engineering process in fit as I nedd the result to fit the scaler
        #Feature engineering
        df_copy=df.copy()
        df_copy, numerical_columns_list = feature_engineering_process (df_copy,self.numerical_columns,self.medians)      
        
        # Scale numerical columns           
        self.scaler.fit(df_copy[numerical_columns_list])

        return self

    def transform(self, df):
        # Handle categorical columns missing values
        df['sex'] = df['sex'].fillna('male')
        df[self.cat_cols] = df[self.cat_cols].fillna('other')            
        
        # Specific replacement for 'tbp_lv_symm_2axis_angle' as 0 mean really missing value
        if 'tbp_lv_symm_2axis_angle' in df.columns:
            df['tbp_lv_symm_2axis_angle'] = df['tbp_lv_symm_2axis_angle'].replace(0, self.medians['tbp_lv_symm_2axis_angle'])        
       
        # Apply one-hot encoding
        X_cat = self.one_hot_encoder.transform(df[self.cat_cols])
        dense_matrix = X_cat.todense()       
        df_encoded = pd.DataFrame(dense_matrix, columns=self.encoded_cols)
        
        #Feature engineering
        df_copy=df.copy()
        df_num, numerical_columns_list = feature_engineering_process (df_copy,self.numerical_columns,self.medians)          
        
        # Scale numerical columns       
        df_num[numerical_columns_list] = self.scaler.transform(df_num[numerical_columns_list])
        
        # Combine encoded and scaled data  
        df_final = pd.concat([df_num, df_encoded], axis=1)
        
        # Add id columns and target back if they exist
        if "isic_id" in df.columns and "patient_id" in df.columns:
            ids = df[["isic_id", "patient_id"]]
            df_final = pd.concat([df_final, ids], axis=1)
        if "target" in df.columns:
            targets = df[["target"]]
            df_final = pd.concat([df_final, targets], axis=1)
        
        return df_final
    

In [17]:
preprocessor = Preprocessor()
preprocessor.fit(df_train_filtered)
df_train_final = preprocessor.transform(df_train_filtered)

/tmp/ipykernel_18/3122119603.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lesion_size_ratio"] = df["tbp_lv_minorAxisMM"] / df["clin_size_long_diam_mm"]
/tmp/ipykernel_18/3122119603.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lesion_shape_index"] = df["tbp_lv_areaMM2"] / (df["tbp_lv_perimeterMM"] ** 2)
/tmp/ipykernel_18/3122119603.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [18]:
len(df_train_final.columns)

79

## Training

### Competition metric

In [19]:
def comp_score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80):
    v_gt = abs(np.asarray(solution.values)-1)
    v_pred = np.array([1.0 - x for x in submission.values])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

### CV validation and shuffling

In [20]:
N_SPLITS = 10
gkf = GroupKFold(n_splits=N_SPLITS)

df_train_final = df_train_final.sample(frac=1).reset_index(drop=True)
df_train_final["fold"] = -1
for idx, (train_idx, val_idx) in enumerate(gkf.split(df_train_final, df_train_final["target"], groups=df_train_final["patient_id"])):
    df_train_final.loc[val_idx, "fold"] = idx
df_train_final

,age_approx,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,...,sex_male,anatom_site_general_anterior torso,anatom_site_general_head/neck,anatom_site_general_lower extremity,anatom_site_general_posterior torso,anatom_site_general_upper extremity,isic_id,patient_id,target,fold
0,0.888687,-0.258416,-0.949418,-0.395963,-0.867580,-1.308590,-1.067123,-1.223959,0.187811,-0.657604,...,1.0,1.0,0.0,0.0,0.0,0.0,ISIC_3974858,IP_0321326,0,8
1,-0.218555,1.090651,-0.651150,-0.500119,-1.081034,-0.732112,-1.104061,-0.803358,-0.402640,-0.017320,...,1.0,0.0,0.0,0.0,1.0,0.0,ISIC_7476537,IP_3927284,0,2
2,0.519606,-0.786562,-0.340742,-0.377168,-0.723872,-0.892798,-0.709525,-0.887973,-0.347257,-0.295247,...,1.0,0.0,0.0,1.0,0.0,0.0,ISIC_4412943,IP_4106376,0,6
3,1.257767,-0.488045,-0.529053,-0.635943,0.560128,0.165371,0.209053,-0.103977,1.012378,0.776163,...,0.0,0.0,0.0,1.0,0.0,0.0,ISIC_4779895,IP_9409053,0,7
4,1.257767,-0.424897,0.398606,0.730885,-0.948540,-0.875623,-0.534655,-0.421042,-1.353235,-1.466109,...,1.0,0.0,0.0,1.0,0.0,0.0,ISIC_7451402,IP_8478332,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395298,-0.956716,0.958615,-0.752026,-1.146924,-1.107958,-1.240037,-1.166767,-1.442463,-0.316575,0.358589,...,0.0,0.0,0.0,1.0,0.0,0.0,ISIC_7682783,IP_5568236,0,9
395299,-0.587636,-0.132121,0.854721,0.207245,-0.130823,0.456535,0.243436,0.413796,-0.896373,0.108905,...,1.0,0.0,0.0,0.0,1.0,0.0,ISIC_8245366,IP_2573038,0,7
395300,1.995929,2.985086,-1.171835,-1.143425,-0.235249,-0.080850,-0.652998,-0.461789,1.062455,1.189559,...,1.0,0.0,0.0,0.0,1.0,0.0,ISIC_7775178,IP_5811816,0,4
395301,-0.218555,-0.201009,0.918259,0.898165,-0.711586,-0.949385,-0.125056,-0.399892,-1.559169,-1.700749,...,0.0,0.0,0.0,1.0,0.0,0.0,ISIC_3064396,IP_7330399,0,8


### training

In [21]:
lgb_params = {
    'objective': 'binary',
    "random_state": 42,
    "n_estimators": 1500,
    'learning_rate': 0.001,
    'bagging_freq': 1,
    'pos_bagging_fraction': 0.75,
    'neg_bagging_fraction': 0.05,
    'feature_fraction': 0.6,
    'lambda_l1': 0.2,
    'lambda_l2': 0.7,
    'num_leaves': 35,
    "min_data_in_leaf": 50,
    "verbosity": -1,
    # "device": "gpu"
    # "extra_trees": True
}

lgb_scores = []
lgb_models = []
train_cols = list(df_train_final.columns)
for c in ["fold", "target","isic_id", "patient_id"]:
    train_cols.remove(c)
for fold in range(N_SPLITS):
    _df_train = df_train_final[df_train_final["fold"] != fold].reset_index(drop=True)
    _df_valid = df_train_final[df_train_final["fold"] == fold].reset_index(drop=True)
    model = LGBMClassifier(**lgb_params)    
    # model = VotingClassifier([(f"lgb_{i}", lgb.LGBMClassifier(random_state=i, **lgb_params)) for i in range(1)], voting="soft")
    model.fit(_df_train[train_cols], _df_train["target"])
    preds = model.predict_proba(_df_valid[train_cols])[:, 1]
    score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
    print(f"fold: {fold} - Partial AUC Score: {score:.5f}")
    lgb_scores.append(score)
    lgb_models.append(model)

fold: 0 - Partial AUC Score: 0.17873
fold: 1 - Partial AUC Score: 0.15767
fold: 2 - Partial AUC Score: 0.15876
fold: 3 - Partial AUC Score: 0.16163
fold: 4 - Partial AUC Score: 0.13094
fold: 5 - Partial AUC Score: 0.18350
fold: 6 - Partial AUC Score: 0.15203
fold: 7 - Partial AUC Score: 0.16949
fold: 8 - Partial AUC Score: 0.09813
fold: 9 - Partial AUC Score: 0.16776


In [22]:
lgbm_score = np.mean(lgb_scores)
print(f"LGBM Score: {lgbm_score:.5f}")

LGBM Score: 0.15586


### Prediction

In [23]:
df_test_final = preprocessor.transform(df_test)
test_cols = list(df_test_final.columns)
for c in ["isic_id", "patient_id", "target"]:
    if c in test_cols:
        test_cols.remove(c)
print(df_test_final[test_cols])
lgb_preds =np.mean([model.predict_proba(df_test_final[test_cols])[:, 1] for model in lgb_models],axis=0)
lgb_preds 

   age_approx  clin_size_long_diam_mm  tbp_lv_A  tbp_lv_Aext  tbp_lv_B  \
0   -0.956716               -0.706192  0.711846     1.452077  0.024677   
1   -1.694878               -0.809525 -0.829749    -1.492132  0.582124   
2    0.519606               -0.442119  1.074854     1.432195  0.419865   

   tbp_lv_Bext  tbp_lv_C  tbp_lv_Cext  tbp_lv_H  tbp_lv_Hext  ...  \
0     0.036119  0.290498     0.573242 -0.621119    -1.331326  ...   
1     0.143826  0.124961    -0.357369  1.334613     1.718607  ...   
2     0.336511  0.734847     0.791126 -0.575095    -1.080711  ...   

   shape_color_consistency  border_length_ratio  age_size_symmetry_index  \
0                -0.937639             0.812669                -0.246080   
1                      NaN             0.040993                -0.626517   
2                -1.086777             0.144205                 0.066274   

   sex_female  sex_male  anatom_site_general_anterior torso  \
0         0.0       1.0                                 0.

/tmp/ipykernel_18/3122119603.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lesion_size_ratio"] = df["tbp_lv_minorAxisMM"] / df["clin_size_long_diam_mm"]
/tmp/ipykernel_18/3122119603.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lesion_shape_index"] = df["tbp_lv_areaMM2"] / (df["tbp_lv_perimeterMM"] ** 2)
/tmp/ipykernel_18/3122119603.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

array([0.00132403, 0.00085546, 0.0026595 ])

### Submission

In [24]:
#df_sub = pd.read_csv("/kaggle/input/isic-2024-challenge/sample_submission.csv")
#df_sub["target"] = lgb_preds
#df_sub.to_csv("submission.csv", index=False)
#df_sub